In [108]:
# set the matplotlib backend so figures can be saved in the background
#TODO: cleanup, lint
import matplotlib
matplotlib.use("Agg")
 
import pathlib
from pathlib import Path
import matplotlib.pyplot as plot
import librosa
import keras
from keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
    
# import the necessary packages
from musicrec.vgg import VGGNet
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras import regularizers
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
from datetime import datetime
import pickle
import cv2
import os
import sys
import tensorflow as tf
import pickle
import json
import re

from keras.models import Sequential

import warnings

#env parameters
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [20]:
# Paramters
data_folder = Path("../../../audio/testfiles/GTZAN/genres/")
output_root = './../../../models/'
output_folder = Path('./output/cvnn.model')
output_model = output_root + '/cnn_dong_model_weights.h5'
output_architecture = output_root + '/cnn_dong_model_architecture.json'
output_whole = output_root + 'cnn_dong_model_whole.h5'
output_best_model = output_root + 'best_model.h5'
output_label = output_root + 'label.pkl'
output_test_paths = output_root + 'test_paths.pkl'
spectogram_folder = Path("./img_data/")
# Duration of songsnippet in seconds
duration = 2.97
# Matplotlib colormap for spectogram
spectogram_cmap = 'binary' 
# Predefined list of genres
pred_genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split() 
epochs = 400
num_segments = 1

es_patience = 10
return_train_and_test = 1

# parameters
sr = 22050 # if sampling rate is different, resample it to this

# parameters for calculating spectrogram in mel scale
fmax = 1500 # maximum frequency considered
fft_window_points = 512
fft_window_dur = fft_window_points * 1.0 / sr # 23ms windows
hop_size = int(fft_window_points/ 2) # 50% overlap between consecutive frames
n_mels = 64

# segment duration
num_fft_windows = 256 # num fft windows per music segment
segment_in_points = num_fft_windows * 255 # number of data points that ensure the spectrogram has size: 64 * 256
segment_dur = segment_in_points * 1.0 / sr

num_genres=10
input_shape=(64, 256, 1)

randomseed = 11
#randomseed = datetime.now()
# Seed for RNG
random.seed(randomseed)


In [21]:
#Get directories of all songs
songs = []
genres = []

for g in data_folder.iterdir():
    genres.append(g.name)
    for i in g.iterdir():
        songs.append(i)

In [22]:
mel_specs = []  
labels = []

#loading with different numbers of segements for data-augmentation
#!Initialize "spectograms" before running
def load_specs(data = songs, num_segments = 1):
    for song in data:
        for i in range(num_segments):
            offset = duration*i
            y, sr = librosa.load(song, mono=True, offset=offset, duration=duration)
            m_sp = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=fft_window_points,
                                                  hop_length=hop_size, n_mels=n_mels,
                                                  fmax=fmax)
        #if m_sp.shape != (128, 128): continue
            mel_specs.append([m_sp])
            label = song.parts[-2]
            labels.append(label)
            spectograms.append( (m_sp, label, song, offset, duration) )
        input_shape = m_sp.shape + (1,)

In [23]:
#shuffle songs to keep segements together
random.shuffle(songs)
print(songs)

[PosixPath('../../../audio/testfiles/GTZAN/genres/blues/blues.00011.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/blues/blues.00064.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/country/country.00009.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/disco/disco.00091.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/classical/classical.00099.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/metal/metal.00002.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/blues/blues.00024.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/country/country.00043.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/rock/rock.00065.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/pop/pop.00002.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/blues/blues.00099.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/disco/disco.00081.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/hiphop/hiphop.00088.wav'), PosixPath('../../../audio

In [24]:
spectograms = []
load_specs(data = songs, num_segments = num_segments)
len(spectograms)

1000

In [25]:
print("Total number of samples: ", len(mel_specs))

#set dynamic input shape
input_shape = np.shape(mel_specs[2]) + (1,)
input_shape = (input_shape[1], input_shape[2], input_shape[3])
print("Input shape: " + str(input_shape))
np.shape(mel_specs)

Total number of samples:  1000
Input shape: (64, 256, 1)


(1000, 1, 64, 256)

In [26]:
batch_size = input_shape[1] # use second dim as batch size, as it varies with duration

In [27]:
imagesize_x = np.shape(mel_specs)[1]
imagesize_y = np.shape(mel_specs)[2]
print("Spectogram dimensions: ", imagesize_x, "x", imagesize_y)

Spectogram dimensions:  1 x 64


In [28]:
testsplit = len(spectograms)*0.7    #70% train-test-split
train = spectograms[:int(testsplit)]
test = spectograms[int(testsplit):]
np.shape(test)

(300, 5)

In [29]:
#TODO: change


X_train, y_train, p_train, offset_train, duration_train = zip(*train)
X_test, y_test, p_test, offset_test, duration_test = zip(*test)

# Reshape for CNN input
X_train = np.array([x.reshape(input_shape) for x in X_train])
X_test = np.array([x.reshape(input_shape) for x in X_test])

# One-Hot encoding for classes
lb = LabelBinarizer()
y_train_bin = lb.fit_transform(y_train)
y_test_bin = lb.transform(y_test)

In [30]:
with open(output_label, 'wb') as f:
    pickle.dump(lb, f)
    
if return_train_and_test == 1:
    r_paths = p_test + (p_train)
    r_offsets = offset_test + (offset_train)
    r_durations = duration_test + (duration_train)
else:
    r_paths = p_test
    r_offsets = offset_test
    r_durations = duration_test
    
r_values = [r_paths, r_offsets, r_durations]

with open(output_test_paths, 'wb') as f:
    pickle.dump(r_values, f)

"""
label_encoder = LabelEncoder()
y_train_int = label_encoder.fit_transform(y_train)
y_test_int = label_encoder.fit_transform(y_test)
#print(integer_encoded)

y_train_bin = np.array(keras.utils.to_categorical(y_train_int, 10))
y_test_bin = np.array(keras.utils.to_categorical(y_test_int, 10))
#print("Encoding of test data:\n", y_test, "=>\n", y_test_int, "=>\n", y_test_bin)
"""
0
len(r_values)

3

In [31]:
y_train = y_train_bin
y_test = y_test_bin

In [32]:
"""

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, 
                         height_shift_range=0.1, shear_range=0.2, 
                         zoom_range=0.2,horizontal_flip=True, 
                         fill_mode="nearest")
 
# initialize our VGG-like Convolutional Neural Network
model = VGGNet.build(width=imagesize_x, height=imagesize_y, depth=1, 
                          classes=len(lb.classes_))
"""
0

0

In [33]:
#old model for urban
def cnn_urban_model_build():
    model = Sequential()

    model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))
    model.add(MaxPooling2D((4, 2), strides=(4, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(48, (5, 5), padding="valid"))
    model.add(MaxPooling2D((4, 2), strides=(4, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(48, (5, 5), padding="valid"))
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dropout(rate=0.5))

    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(rate=0.5))

    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(
        optimizer="Adam",
        loss="categorical_crossentropy",
        metrics=['accuracy'])
    print(model.summary)
    return model

#model for GTZAN
def cnn_dong_model_build():

    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3, 3),
                     activation='relu', kernel_regularizer=regularizers.l2(0.01),
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 4)))

    model.add(Conv2D(64, (3, 5), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(MaxPooling2D(pool_size=(2, 4)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.02)))
    model.add(Dropout(0.3))
    model.add(Dense(num_genres, activation='softmax'))
    #model.compile(loss=keras.losses.categorical_crossentropy,
     #             optimizer=keras.optimizers.Adadelta(decay=1e-6),
      #            metrics=['accuracy'])
    model.compile(
        optimizer="Adam",
        loss="categorical_crossentropy",
        metrics=['accuracy'])
    print(model.summary)
    return model

In [34]:
model = cnn_dong_model_build()

<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x7f0a7421d240>>


In [35]:
callbacks = [EarlyStopping(monitor='val_loss', patience=es_patience),
             ModelCheckpoint(filepath=output_best_model, monitor='val_loss', save_best_only=True)]

H = model.fit(
	x=X_train, 
	y=y_train,
    epochs=epochs,
    callbacks=callbacks,
    batch_size=batch_size,
    validation_data= (X_test, y_test))

score = model.evaluate(x=X_test,y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 700 samples, validate on 300 samples
Epoch 1/400
700/700 [==============================] - 23s 33ms/step - loss: 5.4665 - acc: 0.1129 - val_loss: 4.1543 - val_acc: 0.1400
Epoch 2/400
700/700 [==============================] - 21s 30ms/step - loss: 4.2027 - acc: 0.1257 - val_loss: 4.0705 - val_acc: 0.1233
Epoch 3/400
700/700 [==============================] - 22s 31ms/step - loss: 4.0362 - acc: 0.1314 - val_loss: 3.9781 - val_acc: 0.1133
Epoch 4/400
700/700 [==============================] - 23s 32ms/step - loss: 3.9287 - acc: 0.1543 - val_loss: 3.8842 - val_acc: 0.1233
Epoch 5/400
700/700 [==============================] - 22s 31ms/step - loss: 3.8172 - acc: 0.1486 - val_loss: 3.7659 - val_acc: 0.1367
Epoch 6/400
700/700 [==============================] - 22s 32ms/step - loss: 3.7057 - acc: 0.1729 - val_loss: 3.6545 - val_acc: 0.1600
Epoch 7/400
700/700 [==============================] - 24s 34ms/step - loss: 3.6003 - acc: 0.1871 - val_loss: 3.5717 - val_acc: 0.1600
Epoch 8/4

In [119]:
def evaluate_song(song):
    start = 0
    r = 0
    for i in range(int(30/(duration/2))-1):
        y, sr = librosa.load(song, mono=True, offset=start, duration=duration)
        m_sp = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=fft_window_points,
                                              hop_length=hop_size, n_mels=n_mels,
                                              fmax=fmax)
        m_sp = np.expand_dims(m_sp, 0)
        m_sp = np.expand_dims(m_sp, 3)
        if r == 0:
            prediction = model.predict(m_sp)
            r = 1
        else:
            prediction = (prediction + model.predict(m_sp))/2
        start = start + duration/2
    return prediction
        
def evaluate_batch(batch):
    predictions = []
    for song in batch:
        predictions.append(evaluate_song(song))
    return predictions

def compare_batch(predictions):
    i = 0
    l = lb.inverse_transform(y_test)
    counter = 0
    for prediction in predictions:
        tr_value = str(l[i])
        pr_value = re.sub('[\[\]\']', '', str(lb.classes_[prediction.argmax(axis=1)]))
        print(tr_value + "=>" + pr_value)
        if tr_value == pr_value:
            counter += counter
        i += 1
    acc = counter/len(predictions)*100
    print("Acc: " + str(acc))

In [86]:
predictions = evaluate_batch(p_test)
print(predictions)

[array([[0.09055974, 0.02660247, 0.06033012, 0.07947962, 0.1138649 ,
        0.06822084, 0.04400785, 0.2582229 , 0.16359803, 0.0951135 ]],
      dtype=float32), array([[0.04012785, 0.02661517, 0.0361052 , 0.17746389, 0.27400416,
        0.07494298, 0.04972643, 0.14931718, 0.13589436, 0.03580279]],
      dtype=float32), array([[0.31142777, 0.00592375, 0.03670133, 0.23530048, 0.1082392 ,
        0.0058279 , 0.02304142, 0.07379767, 0.07179305, 0.1279474 ]],
      dtype=float32), array([[0.01918888, 0.00734397, 0.01269281, 0.2345023 , 0.08674303,
        0.00799961, 0.33409303, 0.21652488, 0.07740212, 0.0035094 ]],
      dtype=float32), array([[0.07325957, 0.11116217, 0.11863193, 0.10838106, 0.09150626,
        0.11481668, 0.11873729, 0.10327633, 0.08355772, 0.07667101]],
      dtype=float32), array([[0.10757178, 0.35474104, 0.12090234, 0.03197517, 0.02961368,
        0.19344682, 0.04372039, 0.02262164, 0.04052948, 0.05487763]],
      dtype=float32), array([[0.06954248, 0.13993876, 0.16374

In [120]:
compare_batch(predictions)

reggae=>pop
reggae=>hiphop
hiphop=>blues
pop=>metal
rock=>metal
classical=>classical
hiphop=>hiphop
hiphop=>blues
jazz=>country
rock=>metal
jazz=>classical
hiphop=>blues
rock=>country
reggae=>blues
reggae=>pop
disco=>disco
rock=>rock
reggae=>metal
classical=>classical
hiphop=>hiphop
rock=>country
metal=>blues
disco=>disco
reggae=>pop
metal=>blues
hiphop=>pop
jazz=>country
metal=>metal
metal=>disco
pop=>hiphop
rock=>country
rock=>blues
reggae=>country
blues=>blues
reggae=>disco
jazz=>classical
rock=>blues
rock=>blues
pop=>country
disco=>country
classical=>classical
classical=>classical
blues=>blues
pop=>pop
reggae=>pop
jazz=>country
disco=>classical
reggae=>reggae
rock=>blues
metal=>blues
classical=>classical
classical=>classical
metal=>disco
country=>country
disco=>disco
rock=>country
metal=>blues
metal=>pop
jazz=>jazz
jazz=>metal
country=>rock
blues=>blues
rock=>classical
country=>jazz
metal=>jazz
country=>country
metal=>metal
pop=>hiphop
hiphop=>disco
hiphop=>hiphop
metal=>metal
jazz

In [84]:
# evaluate the network
print("[INFO] evaluating network...")
#predictions = model.predict(X_test, batch_size=batch_size)



print(classification_report(y_test_bin.argmax(axis=1),
                            predictions.argmax(axis=1), 
                            target_names=lb.classes_))
 
# plot the training loss and accuracy
N = np.arange(0, epochs)
plt.style.use("ggplot")
plt.figure()
"""
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (SmallVGGNet)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
"""
0
print(predictions)

[INFO] evaluating network...
[[4.86272052e-02 7.02998862e-02 2.69141346e-02 ... 6.46456480e-02
  1.82405144e-01 4.20449786e-02]
 [1.85398255e-02 1.30622135e-02 8.33014306e-03 ... 1.80129766e-01
  1.26566023e-01 2.23886333e-02]
 [8.71631056e-02 5.77525228e-09 4.33520881e-06 ... 4.23762232e-01
  3.57309580e-01 2.61262851e-03]
 ...
 [8.63931477e-02 4.18305069e-01 1.04098275e-01 ... 2.26575993e-02
  4.42071706e-02 3.97308581e-02]
 [3.28115113e-02 1.39204058e-05 7.26938099e-02 ... 9.90587547e-02
  1.28184129e-02 7.69537330e-01]
 [6.14472665e-02 5.18468730e-02 1.46362841e-01 ... 5.48918499e-03
  1.38488663e-02 1.84170473e-02]]
              precision    recall  f1-score   support

       blues       0.23      0.55      0.32        22
   classical       0.48      0.92      0.63        25
     country       0.25      0.43      0.32        28
       disco       0.55      0.55      0.55        31
      hiphop       0.32      0.31      0.31        26
        jazz       0.53      0.24      0.33   

In [37]:
cm = confusion_matrix(y_test_bin.argmax(axis=1), predictions.argmax(axis=1))

#cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm

array([[12,  1,  1,  0,  1,  2,  0,  2,  0,  3],
       [ 0, 23,  1,  0,  0,  1,  0,  0,  0,  0],
       [ 4,  5, 12,  1,  2,  2,  1,  1,  0,  0],
       [ 1,  1,  0, 17,  4,  1,  0,  3,  2,  2],
       [ 3,  0,  0,  4,  8,  0,  0,  5,  5,  1],
       [ 6,  9,  7,  1,  0,  8,  1,  1,  0,  1],
       [ 5,  1, 11,  2,  2,  0,  5,  4,  0,  3],
       [ 4,  2,  3,  3,  1,  1,  0, 14,  3,  2],
       [ 3,  1,  2,  2,  6,  0,  1,  7,  4,  1],
       [14,  5, 11,  1,  1,  0,  2,  3,  0,  4]])

In [38]:
# Save the weights
#model.save_weights(output_model)

# Save the model architecture
#with open(output_architecture, 'w') as f:
#    f.write(model.to_json())
    
#model.save(output_whole)